# Exploration of data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd

# import numpy as np
# import pyarrow

In [ ]:
#!conda install -c conda-forge pyarrow

## Test script

In [ ]:
import fetch_data as fd

In [ ]:
fd.get_statuses()

## Load 'historique_stations' data

In [ ]:
file_path = os.path.join("..", "data", "historique_stations_2022-04-30_233325.csv")

In [ ]:
def get_df(file_path):
    """ Get dataframe from provided path """
    df = pd.read_csv(
        file_path, 
        parse_dates=[0], 
        header=None, 
        names= ["date", "capacity","available_mechanical","available_electrical", "stationCode", "station_geo","operative"]
    )
    return df

In [ ]:
df = get_df(file_path)
df

### Extract specific stations

In [ ]:
# NB; strings since some station names are not int(!)

ref_ids = [
    #"Molière - République",
    '21209',
    #"Jean Marin Naudin - Stalingrad",
    '22202',
    #"Arthur Auger - Jean Jaurès",
    '21205',
    #"Marne - Germain Dardan"
    '21212',
]

In [ ]:
df[df["stationCode"].isin(ref_ids)]

## Load status data

In [ ]:
status_path = os.path.join("..", "data", "station_status_2022-05-01_080604.csv")

In [ ]:
def get_status_df(file_path):
    """ Get dataframe from provided path """
    df = pd.read_csv(
        file_path, 
        parse_dates=[0], 
        header=None, 
        names= ["date", "station_code", "available_mechanical", "available_electrical", "operative"],
        index_col="date"
    )
    return df

In [ ]:
status_df = get_status_df(status_path)

status_df

## Data cleaning and collection

In [ ]:
def get_historique_file(input_path, has_name=False, has_code=False):
    """ Get 'historique' file """
    
    try:
        col_names = ["date", "capacity","available_mechanical","available_electrical"]
        if has_name:
            col_names.append("stationName")
        if has_code:
            col_names.append("stationCode")
        
        # In any case
        col_names += ["station_geo","operative"]
        
        df = pd.read_csv(input_path, header=None, parse_dates=[0],
            names= col_names,
            index_col="date"
           )
        
        return df
    except KeyError as e:
        msg = "Something wrong in '{}'. Error details:".format(input_path)
        print(msg)
        print(e)

### Process 2022-04-30

In [ ]:
data_path = os.listdir("../data")

process_04_30 = [os.path.join("..", "data", file_name) for file_name in data_path if file_name.startswith("historique_stations_2022-04")]

print(process_04_30)

In [ ]:
# Create full histo df
histo_df = pd.concat([get_historique_file(file_path, has_name=True) for file_path in process_04_30[:5]], axis=0)

histo_df = pd.concat([get_historique_file(file_path, has_name=True, has_code=True) for file_path in process_04_30[5:9]]+[histo_df], axis=0)

histo_df = pd.concat([get_historique_file(file_path, has_code=True) for file_path in process_04_30[9:]]+[histo_df], axis=0)

histo_df.sort_index(inplace=True)

In [ ]:
histo_df.to_parquet("..\data\Summary_2022-04-30.parquet")

In [ ]:
histo_df

In [ ]:
aux_df=pd.read_parquet("..\data\Summary_2022-04-30.parquet")

In [ ]:
aux_df.equals(histo_df)

### Process 2022-05-01

In [ ]:
data_path = os.listdir("../data")

In [ ]:
histo_05_01 = [os.path.join("..", "data", file_name) for file_name in data_path if file_name.startswith("historique_stations_2022-05-01")]

In [ ]:
status_05_01 = [os.path.join("..", "data", file_name) for file_name in data_path if file_name.startswith("station_status_2022-05-01")]

In [ ]:
histo_df = pd.concat([get_historique_file(file_path, has_code=True) for file_path in histo_05_01], axis=0)

histo_df = pd.concat([get_status_df(file_path) for file_path in status_05_01], axis=0)

histo_df.sort_index(inplace=True)

histo_df.to_parquet("..\data\Summary_2022-05-01.parquet")

### Other dates

In [ ]:
import daily_update as du

In [ ]:
for day in ["06", "07", "08", "09"]:
    du.collect_statuses("2022-05-{}".format(day))

In [ ]:
def get_save_status(date_str):
    """ Get and save data of provided date """
    data_path = os.listdir("../data")
    
    prefix_str = "station_status_{}".format(date_str)
    status_day = [os.path.join("..", "data", file_name) for file_name in data_path if file_name.startswith(prefix_str)]
    
    histo_df = pd.concat([get_status_df(file_path) for file_path in status_day], axis=0)
    
    histo_df.sort_index(inplace=True)
    
    save_path = "..\data\Summary_{}.parquet".format(date_str)

    histo_df.to_parquet(save_path)
    
    return histo_df

In [ ]:
histo_df = get_save_status("2022-05-02")

histo_df

In [ ]:
histo_df.index.unique()

In [ ]:
get_historique_file(process_04_30[-1])

In [ ]:
# Manual additional processing
other_paths = ['..\data\historique_stations_2022-04-30_231344.csv', '..\data\historique_stations_2022-04-30_232351.csv', '..\data\historique_stations_2022-04-30_233210.csv', '..\data\historique_stations_2022-04-30_233236.csv']

extra_df = pd.concat([get_historique_file(file_path, extra_cols=True) for file_path in other_paths], axis=0)

In [ ]:
extra_df

In [ ]:
full_histo_df = pd.concat([histo_df, extra_df], axis=0)

In [ ]:
full_histo_df

In [ ]:
input_path = '..\data\historique_stations_2022-04-30_231344.csv'

df = pd.read_csv(input_path, header=None, parse_dates=[0],
    names= ["date", "capacity","available_mechanical","available_electrical", "stationName", "stationCode", "station_geo","operative"],
    index_col="date"
   )

In [ ]:
df

In [ ]:
histo_df[["stationCode", "capacity"]].index.unique()

In [ ]:
date_str = "2022-05-02"

prefix_str = "station_status_{}".format(date_str)
status_day = [os.path.join("..", "data", file_name) for file_name in data_path if file_name.startswith(prefix_str)]

In [ ]:
get_status_df(status_day[-1])